In [3]:
import numpy as np
import pandas as pd

In [13]:
#populationData = pd.read_csv('/Users/danny/Downloads/UNdata_Export_20230714_081108077.csv', dtype=object)
#childrenData = pd.read_csv('/Users/danny/Downloads/UNdata_Export_20230714_054021753.csv')
childrenData = pd.read_csv(r'C:\Users\danny\Downloads\UNdata_Export_20230714_054021753.csv')

In [14]:
#populationData

In [84]:
childrenData

,Country or Area,Year,Area,Sex,Age,Number of children living,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Albania,2011,Total,Female,15 - 19,Total,Census - de jure - complete tabulation,"Final figure, complete",2013.0,3793.0,NaN
1,Albania,2011,Total,Female,15 - 19,0,Census - de jure - complete tabulation,"Final figure, complete",2013.0,120.0,NaN
2,Albania,2011,Total,Female,15 - 19,1,Census - de jure - complete tabulation,"Final figure, complete",2013.0,2919.0,NaN
3,Albania,2011,Total,Female,15 - 19,2,Census - de jure - complete tabulation,"Final figure, complete",2013.0,526.0,NaN
4,Albania,2011,Total,Female,15 - 19,3,Census - de jure - complete tabulation,"Final figure, complete",2013.0,159.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
54237,47,"Data refer to married, widowed and divorced fe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54238,48,Excluding Channel Islands (Guernsey and Jersey...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54239,49,Excluding homeless persons.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54240,50,Excluding Indian jungle population.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
#will be using number of Females by Census (and tabulation) to avoid faulty estimates
# numWomen = populationData[(populationData.get('Area') == 'Total') 
#                           & (populationData.get('Record Type').str.contains('Census')) 
#                           & (populationData.get('Sex') == 'Female')]
# numWomen = numWomen[['Country or Area', 'Year', 'Value']]
# numWomen = numWomen.rename(columns = {'Value': 'Number of Women'})
# print('number of rows: ' + str(numWomen.shape[0]))
# numWomen

In [87]:
#Will use Total because some data points may not have aggregate number of children living 
numChild = childrenData[(childrenData.get('Country or Area') == 'Albania') & (childrenData.get('Area') == 'Total') & (childrenData.get('Age') == '15 +') 
                        & (childrenData.get('Number of children living') != 'Total') & (childrenData.get('Number of children living') != 'Aggregate number of children living')]
numChild['Number of children living'] = numChild['Number of children living'].str.replace('6 +', '6')
numChild['Number of children living'] = numChild['Number of children living'].astype('int32')

# numChild.drop(numChild[numChild['Number of children living'].dtypes == str], inplace = True)
# numChild['Number of children living'] = [i * j for i, j in zip(numChild['Number of children living'], numChild['Number of Women'])] 
numChild = numChild[['Country or Area', 'Year', 'Number of children living', 'Value']] 
numChild = numChild.rename(columns = {'Value': 'Number of Women'})

numChild['Number of Children'] = [i * j for i, j in zip(numChild['Number of children living'], numChild['Number of Women'])] 
numChild = numChild.drop(columns = 'Number of children living')
numChild = numChild.groupby(by=['Country or Area', 'Year']).sum()
numChild = numChild.reset_index()

# print('number of rows: ' + str(numChild.shape[0]))
numChild

C:\Users\danny\AppData\Local\Temp\ipykernel_6096\247167559.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numChild['Number of children living'] = numChild['Number of children living'].str.replace('6 +', '6')
C:\Users\danny\AppData\Local\Temp\ipykernel_6096\247167559.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numChild['Number of children living'] = numChild['Number of children living'].astype('int32')


,Country or Area,Year,Number of Women,Number of Children
0,Albania,2001,793584.0,2459434.0
1,Albania,2011,799159.0,2336289.0


In [20]:
# temp = Albania.merge(numWomen, left_on = ['Country or Area', 'Year'], right_on = ['Country or Area', 'Year'])
# temp = temp.astype({'Number of Children Alive': 'int32', 'Number of Women': 'int32'})
# temp['Number of Children Living Per Woman'] = [i / j for i, j in 
#                                                zip(temp['Number of Children Alive'], temp['Number of Women'])]
# temp